## Imports

In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim

## Read data

In [11]:
# Pfade zu den Verzeichnissen
path_cancer = '/Users/I550949/Desktop/Master Inforamtik/1.Fachsemester/ML/lung_colon_image_set/colon_image_sets/colon_aca'
path_no_cancer = "/Users/I550949/Desktop/Master Inforamtik/1.Fachsemester/ML/lung_colon_image_set/colon_image_sets/colon_n"

# Bilder einlesen: jedes 10. Bild
def load_images(path, label):
    images = []
    for idx, file in enumerate(sorted(os.listdir(path))):
        if idx % 10 == 0 and file.endswith(".jpeg"):
            images.append((os.path.join(path, file), label))
    return images

# Datensätze laden
cancer_images = load_images(path_cancer, 1)  # Label 1 für Krebs
no_cancer_images = load_images(path_no_cancer, 0)  # Label 0 für kein Krebs

# Gesamtdatensatz kombinieren
dataset = cancer_images + no_cancer_images

In [12]:
# Dataset Klasse
class CancerDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# Transformationen (Datenaugmentation und Normalisierung)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [13]:
# Datensatz und DataLoader
cancer_dataset = CancerDataset(dataset, transform=transform)
dataloader = DataLoader(cancer_dataset, batch_size=16, shuffle=True)

## Model

In [17]:
# CNN-Modell
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modell initialisieren
model = SimpleCNN()

# Verlustfunktion und Optimierer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training

In [18]:
# Training
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")

Epoch 1, Loss: 1.019948949889531
Epoch 2, Loss: 0.6300615408117809
Epoch 3, Loss: 0.28644345121251213
Epoch 4, Loss: 0.0533110050055834
Epoch 5, Loss: 0.003554213486264445
Epoch 6, Loss: 0.0007517580869760076
Epoch 7, Loss: 0.00024819323262486177
Epoch 8, Loss: 0.00011902514476484309
Epoch 9, Loss: 4.020649805805454e-05
Epoch 10, Loss: 1.3691764986596975e-05


## Evaluation

In [19]:
# Evaluation
correct = 0
total = 0
with torch.no_grad():
    for images, labels in dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 100.0%
